<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part10_multiquery_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Requirements.txt**

In [ ]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo cohere >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo llama-index >> requirements.txt
!echo pypdf >> requirements.txt
!echo chromadb >> requirements.tx
!echo beautifulsoup4 >> requirements.tx
!echo matplotlib >> requirements.tx
!echo rank_bm25 >> requirements.tx
!echo replicate >> requirements.txt

**Bash/command**

In [ ]:
pip install -r requirements.txt

In [ ]:
%%writefile multiqueryhelper.py

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import cohere
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
#my_key_cohere = os.getenv("cohere_apikey")

my_key_openai="---"
my_key_google="---"
my_key_cohere="---"

llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-pro")
llm_openai = ChatOpenAI(api_key=my_key_openai)
embeddings = OpenAIEmbeddings(api_key=my_key_openai)
cohere_client = cohere.Client(api_key=my_key_cohere)

def generate_multi_query(original_prompt):
    multiquery_prompt = f"""Sen bir yapay zeka asistanısın.

    Bir vektör veri tabanından, kullanıcı sorusuna en fazla benzerlik gösteren dokümanların getirilmesi için, sana verilen kullanıcı girdisinin 3 farklı versiyonunu yazmakla görevlisin.

    Bunu yaparken amacın ise vektörleri karşılaştırırken kullanılan mesafe ölçümlerinin bazı sınırlılıklarını aşmak için, verilen soruyla ilgili birden çok bakış açısı geliştirerek kullanıcıya yardımcı olmak.

    Bu yazacağın alternatif soruları ayrı ayrı satırlarda olacak şekilde yaz.
    Alternatif soruları yazarken bunların 1, 2, 3 gibi numaralandırmalar koyma.

    Kullanıcı girdisi şöyle: {original_prompt}"""

    generated_queries = llm_openai.invoke(input=multiquery_prompt) # chatbot created several questions.

    temp_list = generated_queries.content.strip().split("\n")

    query_list = [original_prompt]
    query_list.extend(temp_list)

    return query_list